<a href="https://colab.research.google.com/github/sahug/ds-bert/blob/main/BERT%20NLP%20-%20Multi%20Class%20Classification%20Using%20BERT%20and%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT NLP - Multi Class Classification Using BERT and Tensorflow**

In [1]:
%pip install -qq transformers

**Import Data**

In [2]:
import pandas as pd

train = pd.read_csv("/content/train.tsv", sep="\t")

In [3]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
y = train["Sentiment"]
x = train["Phrase"]

In [5]:
x

0         A series of escapades demonstrating the adage ...
1         A series of escapades demonstrating the adage ...
2                                                  A series
3                                                         A
4                                                    series
                                ...                        
156055                                            Hearst 's
156056                            forced avuncular chortles
156057                                   avuncular chortles
156058                                            avuncular
156059                                             chortles
Name: Phrase, Length: 156060, dtype: object

In [6]:
y

0         1
1         2
2         2
3         2
4         2
         ..
156055    2
156056    1
156057    3
156058    2
156059    2
Name: Sentiment, Length: 156060, dtype: int64

**Train and Test Split**

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
x_train, x_test, y_train, y_test

(22538                                                cheesy
 99237                                 unintentionally -RRB-
 60377     will need all the luck they can muster just fi...
 128317    somewhere between Sling Blade and South of Hea...
 20776     reminds at every turn of Elizabeth Berkley 's ...
                                 ...                        
 119879                                its trademark villain
 103694    earn her share of the holiday box office pie ,...
 131932    A moving tale of love and destruction in unexp...
 146867    If you love reading and\/or poetry , then by a...
 121958                         and drag audience enthusiasm
 Name: Phrase, Length: 124848, dtype: object,
 95722                                           's as sorry
 147312    Romantic comedy and Dogme 95 filmmaking may se...
 36991                                         of these days
 150211                 flinch from its unsettling prognosis
 140655                             are

In [9]:
y_train.unique()

array([1, 2, 0, 3, 4])

**OHE**

In [10]:
import tensorflow as tf
ohe = tf.keras.layers.CategoryEncoding(num_tokens=len(y_train.unique()), output_mode="one_hot")
y_test = ohe(y_test)
y_train = ohe(y_train)

**Preprocessing**

In [11]:
%pip install -qq tensorflow_hub 
%pip install -qq tensorflow_text

In [21]:
import tensorflow_hub as hub
import tensorflow_text as text

In [25]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [26]:
def pre_processor(example):
  return preprocessor(example)

**Encoder**

In [27]:
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [28]:
def encode_input(preprocessed_text):
  return encoder(preprocessed_text)

**Model**

In [29]:
from tensorflow import keras

In [31]:
inputs = keras.layers.Input(shape=(), dtype=tf.string, name="inputs")
preprocess = pre_processor(inputs)
encode = encode_input(preprocess)

nn1 = keras.layers.Dropout(0.1, name="dropout")(encode["pooled_output"])
nn1 = keras.layers.Dense(5, activation=keras.activations.softmax, name="output")(nn1)

model = keras.Model(inputs=[inputs], outputs=[nn1])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_word_ids':   0           ['inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [32]:
#Compile Model
METRICS = [
           tf.keras.metrics.BinaryAccuracy(name="accuracy"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=METRICS)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2)

Epoch 1/2
   7/3902 [..............................] - ETA: 15:01:51 - loss: 1.8864 - accuracy: 0.7768 - precision: 0.2174 - recall: 0.0446